## UNData API Exercise

In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).

## 1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [5]:
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD?date=2017&per_page=2500&format=json'
response = requests.get(endpoint)

In [7]:
response

<Response [200]>

In [8]:
res = response.json()

In [9]:
len(res[1])

266

In [10]:
res[1][0].keys()

dict_keys(['indicator', 'country', 'countryiso3code', 'date', 'value', 'unit', 'obs_status', 'decimal'])

In [11]:
res[1][263]

{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
  'value': 'GDP per capita, PPP (constant 2021 international $)'},
 'country': {'id': 'YE', 'value': 'Yemen, Rep.'},
 'countryiso3code': 'YEM',
 'date': '2017',
 'value': None,
 'unit': '',
 'obs_status': '',
 'decimal': 0}

In [12]:
un_per_capita = pd.DataFrame()

In [13]:
i = 0
for result in res[1]:
    un_per_capita.loc[i,'country'] = result['country']['value']
    un_per_capita.loc[i,'per_capita'] = result['value']
    i += 1

In [14]:
un_per_capita

,country,per_capita
0,Africa Eastern and Southern,4085.879143
1,Africa Western and Central,4709.645461
2,Arab World,16054.975876
3,Caribbean small states,19132.939321
4,Central Europe and the Baltics,34765.911440
...,...,...
261,Virgin Islands (U.S.),43277.154652
262,West Bank and Gaza,6426.795216
263,"Yemen, Rep.",NaN
264,Zambia,3612.505977


## 2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [16]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN?date=2017&per_page=2500&format=json'
response = requests.get(endpoint)

In [17]:
res = response.json()

In [18]:
res[0]

{'page': 1,
 'pages': 1,
 'per_page': 2500,
 'total': 266,
 'sourceid': '2',
 'lastupdated': '2025-03-24'}

In [19]:
res[1][0].keys()

dict_keys(['indicator', 'country', 'countryiso3code', 'date', 'value', 'unit', 'obs_status', 'decimal'])

In [20]:
res[1][240]

{'indicator': {'id': 'SP.DYN.LE00.IN',
  'value': 'Life expectancy at birth, total (years)'},
 'country': {'id': 'TZ', 'value': 'Tanzania'},
 'countryiso3code': 'TZA',
 'date': '2017',
 'value': 65.999,
 'unit': '',
 'obs_status': '',
 'decimal': 0}

In [21]:
un_leb = pd.DataFrame()

In [22]:
i = 0
for result in res[1]:
    un_leb.loc[i,'country'] = result['country']['value']
    un_leb.loc[i,'life_expectancy'] = result['value']
    i += 1

## 3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [24]:
un = pd.merge(un_per_capita, un_leb, 
              left_on = 'country', right_on = 'country', 
              how = 'inner')

In [25]:
un

,country,per_capita,life_expectancy
0,Africa Eastern and Southern,4085.879143,62.924810
1,Africa Western and Central,4709.645461,56.826634
2,Arab World,16054.975876,71.320870
3,Caribbean small states,19132.939321,72.433246
4,Central Europe and the Baltics,34765.911440,76.927649
...,...,...,...
261,Virgin Islands (U.S.),43277.154652,79.368293
262,West Bank and Gaza,6426.795216,74.833000
263,"Yemen, Rep.",NaN,65.957000
264,Zambia,3612.505977,62.120000


## 4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [27]:
endpoint = 'https://api.worldbank.org/v2/country?date=2017&per_page=2500&format=json'
response = requests.get(endpoint)

In [28]:
res = response.json()

In [29]:
res[0]

{'page': 1, 'pages': 1, 'per_page': '2500', 'total': 296}

In [30]:
res[1][0]

{'id': 'ABW',
 'iso2Code': 'AW',
 'name': 'Aruba',
 'region': {'id': 'LCN',
  'iso2code': 'ZJ',
  'value': 'Latin America & Caribbean '},
 'adminregion': {'id': '', 'iso2code': '', 'value': ''},
 'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
 'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
 'capitalCity': 'Oranjestad',
 'longitude': '-70.0167',
 'latitude': '12.5167'}

In [31]:
un_countries = pd.DataFrame()

In [32]:
i = 0
for result in res[1]:
    un_countries.loc[i,'country'] = result['name']
    un_countries.loc[i,'region'] = result['region']['value']
    un_countries.loc[i,'income_level'] = result['incomeLevel']['value']
    un_countries.loc[i,'capital'] = result['capitalCity']
    un_countries.loc[i,'longitude'] = result['longitude']
    un_countries.loc[i,'latitude'] = result['latitude']
    i += 1

In [33]:
un_countries

,country,region,income_level,capital,longitude,latitude
0,Aruba,Latin America & Caribbean,High income,Oranjestad,-70.0167,12.5167
1,Africa Eastern and Southern,Aggregates,Aggregates,,,
2,Afghanistan,South Asia,Low income,Kabul,69.1761,34.5228
3,Africa,Aggregates,Aggregates,,,
4,Africa Western and Central,Aggregates,Aggregates,,,
...,...,...,...,...,...,...
291,Sub-Saharan Africa excluding South Africa and ...,Aggregates,Aggregates,,,
292,"Yemen, Rep.",Middle East & North Africa,Low income,Sana'a,44.2075,15.352
293,South Africa,Sub-Saharan Africa,Upper middle income,Pretoria,28.1871,-25.746
294,Zambia,Sub-Saharan Africa,Lower middle income,Lusaka,28.2937,-15.3982


In [34]:
un_full = pd.merge(un_countries, un, 
              left_on = 'country', right_on = 'country', 
              how = 'inner')

In [35]:
un_full

,country,region,income_level,capital,longitude,latitude,per_capita,life_expectancy
0,Aruba,Latin America & Caribbean,High income,Oranjestad,-70.0167,12.5167,38558.004558,75.903000
1,Africa Eastern and Southern,Aggregates,Aggregates,,,,4085.879143,62.924810
2,Afghanistan,South Asia,Low income,Kabul,69.1761,34.5228,2952.998916,63.016000
3,Africa Western and Central,Aggregates,Aggregates,,,,4709.645461,56.826634
4,Angola,Sub-Saharan Africa,Lower middle income,Luanda,13.242,-8.81155,9042.117129,61.680000
...,...,...,...,...,...,...,...,...
259,Kosovo,Europe & Central Asia,Upper middle income,Pristina,20.926,42.565,10856.290207,78.783000
260,"Yemen, Rep.",Middle East & North Africa,Low income,Sana'a,44.2075,15.352,NaN,65.957000
261,South Africa,Sub-Saharan Africa,Upper middle income,Pretoria,28.1871,-25.746,14591.436676,65.402000
262,Zambia,Sub-Saharan Africa,Lower middle income,Lusaka,28.2937,-15.3982,3612.505977,62.120000


## **Bonus Questions:** After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

## 1. Adjust your request so that it returns data just for the United States.

In [38]:
endpoint = 'https://api.worldbank.org/v2/country/usa/indicator/NY.GDP.PCAP.PP.KD?date=2017&per_page=2500&format=json'
requests.get(endpoint).json()

[{'page': 1,
  'pages': 1,
  'per_page': 2500,
  'total': 1,
  'sourceid': '2',
  'lastupdated': '2025-03-24'},
 [{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'US', 'value': 'United States'},
   'countryiso3code': 'USA',
   'date': '2017',
   'value': 66458.0233734963,
   'unit': '',
   'obs_status': '',
   'decimal': 0}]]

## 2. Adjust your request so that it returns data just for the United States for the year 2021.

In [40]:
endpoint = 'https://api.worldbank.org/v2/country/usa/indicator/NY.GDP.PCAP.PP.KD?date=2021&per_page=2500&format=json'
requests.get(endpoint).json()

[{'page': 1,
  'pages': 1,
  'per_page': 2500,
  'total': 1,
  'sourceid': '2',
  'lastupdated': '2025-03-24'},
 [{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'US', 'value': 'United States'},
   'countryiso3code': 'USA',
   'date': '2021',
   'value': 71318.3073592183,
   'unit': '',
   'obs_status': '',
   'decimal': 0}]]

## 3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

In [42]:
endpoint = 'https://api.worldbank.org/v2/country/usa/indicator/NY.GDP.PCAP.PP.KD?date=2000:2021&per_page=2500&format=json'
requests.get(endpoint).json()

[{'page': 1,
  'pages': 1,
  'per_page': 2500,
  'total': 22,
  'sourceid': '2',
  'lastupdated': '2025-03-24'},
 [{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'US', 'value': 'United States'},
   'countryiso3code': 'USA',
   'date': '2021',
   'value': 71318.3073592183,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'US', 'value': 'United States'},
   'countryiso3code': 'USA',
   'date': '2020',
   'value': 67352.3871392801,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'US', 'value': 'United States'},
   'countryiso3code': 'USA',
   'date': '2019',
   'value': 69511.7634409067,
   'unit': '',
   'obs_status': '',
   'deci

## 4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

In [44]:
endpoint = 'https://api.worldbank.org/v2/country/can;usa/indicator/NY.GDP.PCAP.PP.KD?date=2000:2021&per_page=2500&format=json'
requests.get(endpoint).json()

[{'page': 1,
  'pages': 1,
  'per_page': 2500,
  'total': 44,
  'sourceid': '2',
  'lastupdated': '2025-03-24'},
 [{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'CA', 'value': 'Canada'},
   'countryiso3code': 'CAN',
   'date': '2021',
   'value': 56547.8005608561,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'CA', 'value': 'Canada'},
   'countryiso3code': 'CAN',
   'date': '2020',
   'value': 54006.5844302454,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'CA', 'value': 'Canada'},
   'countryiso3code': 'CAN',
   'date': '2019',
   'value': 57491.9820463203,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicat

## 5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

In [46]:
# I'll combine this exercise with the next question

## 6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.

In [48]:
endpoint = 'https://api.worldbank.org/v2/indicator?format=json&per_page=250'
res = requests.get(endpoint).json()

In [49]:
res[0]

{'page': 1, 'pages': 92, 'per_page': '250', 'total': 22807}

In [50]:
res[1][0]

{'id': '1.0.HCount.1.90usd',
 'name': 'Poverty Headcount ($1.90 a day)',
 'unit': '',
 'source': {'id': '37', 'value': 'LAC Equity Lab'},
 'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2011 PPP) below the poverty line.',
 'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
 'topics': [{'id': '11', 'value': 'Poverty '}]}

In [51]:
len(res[1])

250

In [52]:
pages = res[0]['pages']

In [53]:
pages

92

In [54]:
import time

In [55]:
        endpoint = 'https://api.worldbank.org/v2/indicator?format=json&per_page=250&page=' + str(i)
        res = requests.get(endpoint).json()
        for indicator in res[1]:
            print(indicator['name'])

In [56]:
indicators = pd.DataFrame(columns=['id','name'])

In [57]:
for i in range(1,pages+1) :
    print('Extracting page ' + str(i) )
    time.sleep(3)
    
    endpoint = 'https://api.worldbank.org/v2/indicator?format=json&per_page=250&page=' + str(i)
    res = requests.get(endpoint).json()
    
    for indicator in res[1]:
        id = indicator['id']
        name = indicator['name']
        new_row = {'id': id, 'name': name}
        indicators.loc[len(indicators)] = new_row

Extracting page 1
Extracting page 2
Extracting page 3
Extracting page 4
Extracting page 5
Extracting page 6
Extracting page 7
Extracting page 8
Extracting page 9
Extracting page 10
Extracting page 11
Extracting page 12
Extracting page 13
Extracting page 14
Extracting page 15
Extracting page 16
Extracting page 17
Extracting page 18
Extracting page 19
Extracting page 20
Extracting page 21
Extracting page 22
Extracting page 23
Extracting page 24
Extracting page 25
Extracting page 26
Extracting page 27
Extracting page 28
Extracting page 29
Extracting page 30
Extracting page 31
Extracting page 32
Extracting page 33
Extracting page 34
Extracting page 35
Extracting page 36
Extracting page 37
Extracting page 38
Extracting page 39
Extracting page 40
Extracting page 41
Extracting page 42
Extracting page 43
Extracting page 44
Extracting page 45
Extracting page 46
Extracting page 47
Extracting page 48
Extracting page 49
Extracting page 50
Extracting page 51
Extracting page 52
Extracting page 53
Ex

In [58]:
indicators['potential_match'] = False

#why doesn't this work???
#indicators['potential_match'] = 'Poverty' in indicators['name']
#Try this instead...
for index, row in indicators.iterrows() :
    indicators.loc[index, 'potential_match'] = 'Public Expenditure on Education' in row['name']
        # Public Expenditure on Education (% GDP)

indicators[indicators['potential_match']]

,id,name,potential_match
20557,SE.XPD.EDUC.ZS,Public Expenditure on Education (% GDP),True


^ Ah!  There is an extra space before the '(% GDP)'!

In [60]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SE.XPD.TOTL.GD.ZS?date=2017&format=json'